In [145]:
import pylab
import imageio
import pandas as pd
from pathlib import Path
import numpy as np
from collections import defaultdict
VID_DATA_DIR = '/mnt/ssd500/hungbnt/liveness_detection/zalo/train/videos'
IMG_DATA_DIR = '/mnt/ssd500/hungbnt/liveness_detection/zalo/train/images'
DF_LABEL_PATH = '/mnt/ssd500/hungbnt/liveness_detection/zalo/train/label.csv'
DF_SAVE_PATH = 'train.csv'
NFRAMES_EXTRACTED = 5 #per video
IMG_EXT = ".jpg" #to save

In [146]:
pvid_data_dir = Path(VID_DATA_DIR)
pvid_paths = pvid_data_dir.glob('*.mp4')
lvid_paths = list(pvid_paths)
len(lvid_paths), lvid_paths[0]

(1168,
 PosixPath('/mnt/ssd500/hungbnt/liveness_detection/zalo/train/videos/76.mp4'))

In [147]:
# print(dir(limg_paths[0]))
# pvid_data_dir.joinpath('test.mp4')
pimg_data_dir = Path(IMG_DATA_DIR)

In [148]:
df_label = pd.read_csv(DF_LABEL_PATH, index_col=0)
len(df_label), df_label.head()

(1168,        liveness_score
 fname                
 1.mp4               0
 2.mp4               1
 3.mp4               1
 5.mp4               0
 7.mp4               1)

In [149]:
def gen_img_name_by_frame_index(i_frame:int, n_zeros=3):
    return str(10**n_zeros + i_frame)[1:] + IMG_EXT# 1 -> 1001 -> 001
    
print(gen_img_name_by_frame_index(1))
    

001.jpg


In [150]:
ddata = defaultdict(list)
for i in range(1,3): 
    filename = lvid_paths[i]
    img_dir = pimg_data_dir.joinpath(filename.stem)
    img_dir.mkdir(parents=True, exist_ok=True) #new folder to store img
    vid = imageio.get_reader(filename,  'ffmpeg')
    nframes = vid.count_frames()
    j_frames = np.linspace(0,nframes-1,NFRAMES_EXTRACTED, dtype=int)
    for j in j_frames:
        img_path = img_dir.joinpath(gen_img_name_by_frame_index(j))
        # imageio.imsave(img_path,vid.get_data(j))
        ddata['rgb_path'].append(str(img_path))
        ddata['video_id'].append(filename.stem)
        ddata['label'].append(df_label.loc[lvid_paths[i].name][0])


In [151]:
df_train = pd.DataFrame.from_dict(ddata)
df_train

,rgb_path,video_id,label
0,/mnt/ssd500/hungbnt/liveness_detection/zalo/tr...,526,0
1,/mnt/ssd500/hungbnt/liveness_detection/zalo/tr...,526,0
2,/mnt/ssd500/hungbnt/liveness_detection/zalo/tr...,526,0
3,/mnt/ssd500/hungbnt/liveness_detection/zalo/tr...,526,0
4,/mnt/ssd500/hungbnt/liveness_detection/zalo/tr...,526,0
5,/mnt/ssd500/hungbnt/liveness_detection/zalo/tr...,988,1
6,/mnt/ssd500/hungbnt/liveness_detection/zalo/tr...,988,1
7,/mnt/ssd500/hungbnt/liveness_detection/zalo/tr...,988,1
8,/mnt/ssd500/hungbnt/liveness_detection/zalo/tr...,988,1
9,/mnt/ssd500/hungbnt/liveness_detection/zalo/tr...,988,1


In [153]:
df_train.to_csv('train_zalo.csv', index=False)